<a href="https://colab.research.google.com/github/vanderbilt-data-science/lo-achievement/blob/update_UI_with_prompts/gradio_application.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Creating an User Interface for Self-Study

In this notebook, we define functionality to assist with self-study. The code is to be deployed on Huggingface Spaces.

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
#| default_exp UIBaseComponents

First, we'll start by loading our own libraries. Keep in mind that if you're on Colab, you need to replace "token" below with your GitHub token.

In [ ]:
# run this code if you're using Google Colab or don't have these packages installed in your computing environment
#! pip install pip install git+https://ghusername:<token>@github.com/vanderbilt-data-science/lo-achievement.git

The following is needed because the notebook has been moved into a subfolder. This is how we're able to actually import our modules of interest.

In [ ]:
import os
import sys

# get parent of current working directory
parent_dir = os.path.dirname(os.getcwd())

#append to path
sys.path.append(parent_dir)

# modules for testing only
from collections import namedtuple
from getpass import getpass


In [ ]:
#| export
import gradio as gr
from functools import partial
import pandas as pd
import os

from ai_classroom_suite.PromptInteractionBase import *
from ai_classroom_suite.SelfStudyPrompts import *
from ai_classroom_suite.MediaVectorStores import *

# Helper Functions for Source Definitions

We'll first start by defining some helper functions to assist with the source definitions. These functions will be used to populate the source area of the UI.

In [ ]:
### Source Display ###

def format_source_reference(document, index):
  """Return a HTML element which contains the `document` info and can be
  referenced by the `index`."""
  source_filepath, source_filename = os.path.split(document.metadata['source'])
  return  f"<p id='source{index}'>[{index+1}] <em>{source_filename}</em><br>...{document.page_content}...</p>"

def format_source_citation(index):
  """Return a Markdown element which references the element with the given
  `index`."""
  return f"[{index+1}](#source{index})"

def update_source_list(added_sources):
  """Add any new sources to the list `srcs`; return a list of integers
  containing the index references for each of the given sources in `sources`."""
  source_indices = []
  for source in added_sources:
    if source not in srcs: srcs.append(source)
    source_indices.append(srcs.index(source))
  return source_indices

def get_source_display_updates(sources=None):
  """Dynamically update the output for the given list of components to show
  the items as contained within the documents in `sources`.
  See https://github.com/gradio-app/gradio/issues/2066 for what I'm copying."""
  # TODO Currently displays the first 100 sources only, which could be an issue
  # TODO Display an alert when the source display is being cut off (over 100
  # sources are stored)
  update_list = []
  for i in range(num_sources):
    if i < len(sources):
      source_text = format_source_reference(sources[i], i)
      update_list.append(gr.update(visible=True, value=source_text))
    else:
      update_list.append(gr.update(visible=False))
  return update_list

# Downloading The Conversation
Now, we'll create the ability to download the data. We'll allow this to be downloaded in several different formats.

In [ ]:
#| export

def save_chatbot_dialogue(chat_tutor, save_type):

    formatted_convo = pd.DataFrame(chat_tutor.conversation_memory, columns=['user', 'chatbot'])

    output_fname = f'tutoring_conversation.{save_type}'

    if save_type == 'csv':
        formatted_convo.to_csv(output_fname, index=False)
    elif save_type == 'json':
        formatted_convo.to_json(output_fname, orient='records')
    elif save_type == 'txt':
        temp = formatted_convo.apply(lambda x: 'User: {0}\nAI: {1}'.format(x[0], x[1]), axis=1)
        temp = '\n\n'.join(temp.tolist())
        with open(output_fname, 'w') as f:
            f.write(temp)
    else:
        gr.update(value=None, visible=False)
    
    return gr.update(value=output_fname, visible=True)

save_pdf = partial(save_chatbot_dialogue, save_type='pdf')
save_json = partial(save_chatbot_dialogue, save_type='json')
save_txt = partial(save_chatbot_dialogue, save_type='txt')
save_csv = partial(save_chatbot_dialogue, save_type='csv')


# Interface Helpers and Functionality
In the next section, we'll create some helper functions and classes to make sure we're able to create the interface we want, and that it behaves nicely. Further description of each piece will be defined below.

## State Class: `SlightlyDelusionalTutor`

We create this class to help us to maintain the state of the gradio application. This containerization helps us not to have to use global variables. Any global variables created outside of the Blocks definitions [unfortunately belong to all the running instances (in other words, across users.)](https://www.gradio.app/guides/interface-state) Instead, we choose to use a class which contains all of the relevant database, vector store, LLM, chain, memory, and source information. In the definition of the interface below, you'll see that we pass this in and out of functions, using the tutor to contain the state of the app, as well as enabling access to important components of the app.

In [ ]:
#| export
class SlightlyDelusionalTutor:
    # create basic initialization function
    def __init__(self, model_name = None):

        # create default model name
        if model_name is None:
            self.model_name = 'gpt-3.5-turbo-16k'

        self.chat_llm = None
        self.tutor_chain = None
        self.vector_store = None
        self.vs_retriever = None
        self.conversation_memory = []
        self.sources_memory = []
        self.flattened_conversation = ''
        self.api_key_valid = False
        self.learning_objectives = None
        self.openai_auth = ''
        self.system_message = ''
    
    def set_system_message(self, message):
        self.system_message = message
    
    def embed_key(self, openai_api_key):
        if not openai_api_key:
            return None
        
        # Otherwise, update key
        os.environ["OPENAI_API_KEY"] = openai_api_key
        
        #update tutor
        self.openai_auth = openai_api_key

        if not self.api_key_valid:
            self.initialize_llm()
    
    def initialize_llm(self):

        if self.openai_auth:
            try:
                self.chat_llm = create_model(self.model_name, openai_api_key = self.openai_auth)
                self.api_key_valid = True
            except Exception as e:
                print(e)
                self.api_key_valid = False
        else:
            print("Please provide an OpenAI API key and press Enter.")
    
    def initialize_basic_model(self, openai_auth=None, **mdl_chain_kwargs):
        # get the key
        if openai_auth is None:
            openai_auth = os.environ.get("OPENAI_API_KEY", None)
            # we need to create some kind of warning here

        # create the tutor chain
        if not self.api_key_valid or not self.openai_auth:
            self.embed_key(openai_auth)
        
        if self.vs_retriever is not None:
            if 'retriever' in mdl_chain_kwargs:
                mdl_chain_kwargs['retriever'] = self.vs_retriever
                mdl_chain_kwargs['mdl'] = self.chat_llm
            self.tutor_chain = create_tutor_mdl_chain(**mdl_chain_kwargs)
        else:
            self.tutor_chain = self.chat_llm
        
    def add_user_message(self, user_message):
        self.conversation_memory.append([user_message, None])
        self.flattened_conversation = self.flattened_conversation + '\n\n' + 'User: ' + user_message
    
    def _update_chat_history(self, tutor_message):
        self.conversation_memory[-1] = (self.conversation_memory[-1][0], tutor_message['answer'])
        self.flattened_conversation = self.flattened_conversation + '\nAI: ' + tutor_message['answer']
        self.sources_memory.append(tutor_message['source_documents'])

    def get_direct_tutor_reply(self):
        if not self.conversation_memory:
            return "Please type something to start the conversation."
        
        # get answer
        message_with_history = self.conversation_memory[-1][0] + '\n\n\n So far, our conversation has been: \n'
        if self.system_message:
            tutor_message = get_direct_answer(self.tutor_chain, human_prompt=message_with_history, system_prompt = self.system_message)
        else:
            tutor_message = get_direct_answer(self.tutor_chain, human_prompt=message_with_history)

        # add tutor message to conversation memory
        self.conversation_memory[-1][1] = tutor_message
        self.flattened_conversation = self.flattened_conversation + '\nAI: ' + tutor_message

    def get_tutor_reply(self, **input_kwargs):

        if not self.conversation_memory:
            return "Please type something to start the conversation."

        # if this is the first interaction, then we need to use the full prompt
        if len(self.conversation_memory) <= 1:
            input_kwargs['chat_history'] = []
            input_kwargs['question'] = ''
        else:
            input_kwargs['chat_history'] = self.conversation_memory[:-1]
            input_kwargs['question'] = self.conversation_memory[-1][0]

        tutor_message = get_tutoring_answer('See provided documents and question for more information.',
                                     self.tutor_chain,
                                     assessment_request = self.conversation_memory[-1][0], #this will not be used after the first interaction
                                     learning_objectives = self.learning_objectives,
                                     return_dict=True,
                                     input_kwargs=input_kwargs)

        # add tutor message to conversation memory
        self._update_chat_history(tutor_message)
        
        #print(self.flattened_conversation, '\n\n')
    
    def get_sources_memory(self):
        # retrieve last source
        last_sources = self.sources_memory[-1]

        # get page_content keyword from last_sources
        doc_contents = ['Source ' + str(ind+1) + '\n"' + doc.page_content + '"\n\n' for ind, doc in enumerate(last_sources)]
        doc_contents = ''.join(doc_contents)

        return doc_contents
    
    def forget_conversation(self):
        self.conversation_memory = []
        self.sources_memory = []
        self.flattened_conversation = ''

## Callbacks
### embed_key

This function is called whenever the API textbox either has someone press enter (`submit` event) or when the textbox loses focus (`blur` event). This allows us to capture the key regardless of whether they press enter or not.

In [ ]:
#| export
def embed_key(openai_api_key, chat_tutor):
  chat_tutor.embed_key(openai_api_key)
  return chat_tutor

In [ ]:
#| export
def initialize_basic_model(chat_tutor, openai_auth=None, **mdl_chain_kwargs):
    chat_tutor.initialize_basic_model(openai_auth, **mdl_chain_kwargs)

    return chat_tutor

### Start Studying with Tutor
The `create_reference_store` function is used essentially to initialize the tutor with the vector store, retriever, and the RetrievalQAWithSources chain. This function is called whenever the user clicks "Starting Studying with Tutor" button after providing any text or stores to be used.

In [ ]:
#| export
def create_reference_store(chat_tutor, vs_button, text_cp, upload_files, reference_vs, openai_auth, learning_objs):

    text_segs = []
    upload_segs = []

    print('I just got called.')
    
    if reference_vs:
        raise NotImplementedError("Reference Vector Stores are not yet implemented")
    
    if text_cp.strip():
        text_segs = get_document_segments(text_cp, 'text', chunk_size=700, chunk_overlap=100)
        [doc.metadata.update({'source':'text box'}) for doc in text_segs];
    
    if upload_files:
        upload_fnames = [f.name for f in upload_files]
        upload_segs = get_document_segments(upload_fnames, 'file', chunk_size=700, chunk_overlap=100)
    
    # get the full list of everything
    all_segs = text_segs + upload_segs

    # store learning objectives
    chat_tutor.learning_objectives = learning_objs

    # create the vector store and update tutor
    vs_db, vs_retriever = create_local_vector_store(all_segs, search_kwargs={"k": 2})
    chat_tutor.vector_store = vs_db
    chat_tutor.vs_retriever = vs_retriever

    # create the tutor chain
    chat_tutor = initialize_basic_model(chat_tutor,
                                        openai_auth=openai_auth,
                                        kind="conversational",
                                        mdl=chat_tutor.chat_llm,
                                        retriever = vs_retriever,
                                        return_source_documents=True)

    # return the story
    return chat_tutor, gr.update(interactive=True, value='Tutor Initialized!')

### A small test suite for reference store, etc. debugging
Occasionally, this code goes wrong. The code below is used for testing in such cases.

In [ ]:
# Setup OpenAI API Key (uncomment to use)
#os.environ['OPENAI_API_KEY'] = getpass()

# Create an instance of the class
#mdl_test = SlightlyDelusionalTutor()

# Define a namedtuple called 'Files' with a field named 'name'
#Files = namedtuple('Files', ['name'])
#up_files = [Files(name='../roadnottaken.txt')]

#outtest, _ = create_reference_store(mdl_test, '', '', up_files, None, os.environ['OPENAI_API_KEY'], 'Some objectives')

In [ ]:
# Add a user message the same way we would normally
#outtest.add_user_message('The road not taken was something interesting.')

# get a tutor reply
#outtest.get_tutor_reply()

# see the tutor reply
#outtest.conversation_memory[-1][1]

This result doesn't look very good but hopefully that won't be an issue when the code is actually running.

### Getting help with prompts
The following `prompt_select` callback is used when the user wants a little help with creating a prompt. As you can see below, the prompt is directly created based on the user's selections. This prompt immediately fills the prompt textbox, as well as begins the conversation in the chat window.

In [ ]:
#| export
### Gradio Called Functions ###

def prompt_select(selection, number, length):
  if selection == "Random":
    prompt = f"Please design a {number} question quiz based on the context provided and the inputted learning objectives (if applicable). The types of questions should be randomized (including multiple choice, short answer, true/false, short answer, etc.).  Provide one question at a time, and wait for my response before providing me with feedback. Again, while the quiz may ask for multiple questions, you should only provide 1 question in you initial response. Do not include the answer in your response. If I get an answer wrong, provide me with an explanation of why it was incorrect, and then give me additional chances to respond until I get the correct choice. Explain why the correct choice is right."
  elif selection == "Fill in the Blank":
    prompt = f"Create a {number} question fill in the blank quiz refrencing the context provided. The quiz should reflect the learning objectives (if inputted). The 'blank' part of the question should appear as '________'. The answers should reflect what word(s) should go in the blank an accurate statement. An example is the follow: 'The author of the article is ______.' The question should be a statement. Provide one question at a time, and wait for my response before providing me with feedback. Again, while the quiz may ask for multiple questions, you should only provide ONE question in you initial response. Do not include the answer in your response. If I get an answer wrong, provide me with an explanation of why it was incorrect,and then give me additional chances to respond until I get the correct choice. Explain why the correct choice is right."
  elif selection == "Short Answer":
    prompt = f"Please design a {number} question quiz about which reflects the learning objectives (if inputted).  The questions should be short answer. Expect the correct answers to be {length} sentences long. Provide one question at a time, and wait for my response before providing me with feedback. Again, while the quiz may ask for multiple questions, you should only provide ONE question in you initial response. Do not include the answer in your response. If I get an answer wrong, provide me with an explanation of why it was incorrect, and then give me additional chances to respond until I get the correct choice. Explain why the correct answer is right."
  else:
    prompt = f"Please design a {number} question {selection.lower()} quiz based on the context provided and the inputted learning objectives (if applicable). Provide one question at a time, and wait for my response before providing me with feedback. Again, while the quiz may ask for multiple questions, you should only provide 1 question in you initial response. Do not include the answer in your response. If I get an answer wrong, provide me with an explanation of why it was incorrect, and then give me additional chances to respond until I get the correct choice. Explain why the correct choice is right."
  return prompt, prompt
    

### Chatting with the Tutor
The following functions are callbacks for the ChatBot functionality, where the user enters their information into a text field, presses enter (or clicks an associated button), and then receives a response. Again, the State Class above keeps track of the entire state of the app, including the conversational history.

The conversational history is currently stored as a list of 2-element lists, where the first element is the user's input, and the second element is the tutor's response. This chat history in this format is actually used as an input to the chatbot, and our stored state facilitates this.

Additionally, within our class, you will see that sending a user message creates a new list-like element in the history list. Whenever we want to get the tutor's response, we get the response and record it as the second element of this conversation entry.

In [ ]:
#| export
### Chatbot Functions ###

def add_user_message(user_message, chat_tutor):
  """Display user message and update chat history to include it.
  Also disables user text input until bot is finished (call to reenable_chat())
  See https://gradio.app/creating-a-chatbot/"""
  chat_tutor.add_user_message(user_message)
  return gr.update(value="", interactive=False), chat_tutor.conversation_memory, chat_tutor

def get_tutor_reply(chat_tutor):
  chat_tutor.get_tutor_reply()
  return gr.update(value="", interactive=True), gr.update(visible=True, value=chat_tutor.get_sources_memory()), chat_tutor.conversation_memory, chat_tutor

num_sources = 2

In [ ]:
#| export
def disable_until_done(obj_in):
    return gr.update(interactive=False)

# The User Interface
Below, we put all of this information together with the actual formatting of the user interface.

In [ ]:
#| export
# add theme information (see. gr.themes.builder())
gr_study_tutor_theme = gr.themes.Soft(
    primary_hue="slate",
).set(
    body_background_fill='*primary_500',
    body_text_color_subdued='*neutral_500',
    body_text_color_subdued_dark='*neutral_500',
    background_fill_primary='*neutral_500',
    background_fill_secondary='*primary_400',
    prose_header_text_weight='700',
    block_background_fill='*primary_300',
    block_border_width='5px',
    block_title_text_size='*text_lg',
    #form_gap_width='*layout_gap',
    panel_border_width='*block_border_width',
    panel_border_width_dark='*block_border_width',
    section_header_text_size='*text_lg'
)

In [ ]:
#| export
# See https://gradio.app/custom-CSS-and-JS/
css="""
#asheader { font-size: 32px !important; font-weight: 800 !important; margin: 0.67em 0; }
#sources-container {
  overflow: scroll !important; /* Needs to override default formatting */
  /*max-height: 20em; */ /* Arbitrary value */
}
#sources-container > div { padding-bottom: 1em !important; /* Arbitrary value */ }
.short-height > * > * { min-height: 0 !important; }
.translucent { opacity: 0.5; }
.textbox_label { padding-bottom: .5em; }
"""
#srcs = [] # Reset sources (db and qa are kept the same for ease of testing)

with gr.Blocks(css=css, analytics_enabled=False, theme=gr_study_tutor_theme) as FullStudyApp:

  #initialize tutor (with state)
  study_tutor = gr.State(SlightlyDelusionalTutor())

  # Title
  gr.Markdown("# Studying with a Slightly Delusional Tutor")

  # API Authentication functionality
  with gr.Box():
    gr.Markdown("### OpenAI API Key ")
    gr.HTML("""<span>Embed your OpenAI API key below; if you haven't created one already, visit
      <a href="https://platform.openai.com/account/api-keys">platform.openai.com/account/api-keys</a>
    to sign up for an account and get your personal API key</span>""",
            elem_classes="textbox_label")
    api_input = gr.Textbox(show_label=False, type="password", container=False, autofocus=True,
                      placeholder="●●●●●●●●●●●●●●●●●", value='')
    api_input.submit(fn=embed_key, inputs=[api_input, study_tutor], outputs=study_tutor)
    api_input.blur(fn=embed_key, inputs=[api_input, study_tutor], outputs=study_tutor)

  # Reference document functionality (building vector stores)
  with gr.Box():
    gr.Markdown("### Add Reference Documents")
    # TODO Add entry for path to vector store (should be disabled for now)
    with gr.Column():
      with gr.Row(equal_height=True):
        text_input = gr.TextArea(label='Copy and paste your text below',
                                  lines=2)

        file_input = gr.Files(label="Load a .txt or .pdf file",
                              file_types=['.pdf', '.txt'], type="file",
                              elem_classes="short-height")

        instructor_input = gr.TextArea(label='Enter vector store URL, if given by instructor (WIP)', value='',
                                  lines=2, interactive=False, elem_classes="translucent")
      
      # Adding the learning objectives
      learning_objectives = gr.Textbox(label='What topic or concept are you trying to learn more about?',
                                      placeholder='e.g., Understanding the best practices of prompting for LLMs', value='')
    
    # Adding the button to submit all of the settings and create the Chat Tutor Chain.
      vs_build_button = gr.Button(value = 'Start Studying with Your Tutor!', scale=1)
      
      vs_build_button.click(disable_until_done, vs_build_button, vs_build_button, queue=True) \
        .then(create_reference_store, [study_tutor, vs_build_button, text_input, file_input, instructor_input, api_input, learning_objectives],
                            [study_tutor, vs_build_button], queue=False)

  

  # Premade question prompts
  with gr.Box():
    gr.Markdown("""## Use Premade Prompts
                Need some help getting started with your prompt? Use the dropdown below to help you get started.
                Otherwise, you can hop to the next section and begin directly interacting with your tutor!""")
    with gr.Accordion("Help me create an interactive prompt", open=False):
      gr.Markdown("""
      ### Generating a prompt
      Select your type and number of desired questions. Click "Generate Prompt" to get your premade prompt,
      and then "Insert Prompt into Chat" to copy the text into the chat interface below. \
      You can also copy the prompt using the icon in the upper right corner and paste directly into the input box when interacting with the model.
      """)
      with gr.Row():
        with gr.Column():
          question_type = gr.Dropdown(["Multiple Choice", "True or False", "Short Answer", "Fill in the Blank", "Random"], label="Question Type")
          number_of_questions = gr.Textbox(label="Enter desired number of questions")
          sa_desired_length = gr.Dropdown(["1-2", "3-4", "5-6", "6 or more"], label = "For short answer questions only, choose the desired sentence length for answers. The default value is 1-2 sentences.")
        with gr.Column():
          prompt_button = gr.Button("Generate Prompt")
          premade_prompt_output = gr.Textbox(label="Generated prompt (save or copy)", show_copy_button=True)


  # Chatbot interface
  with gr.Box():
    gr.Markdown("## Chat with Your Tutor")
    with gr.Row(equal_height=True):
      with gr.Column(scale=2):
        chatbot = gr.Chatbot()
        with gr.Row():
          user_chat_input = gr.Textbox(label="User input", scale=9)
          user_chat_submit = gr.Button("Ask/answer model", scale=1)

      # sources
      with gr.Box(elem_id="sources-container", scale=1):
        # TODO: Display document sources in a nicer format?
        gr.HTML(value="<h3 id='sources'>Referenced Sources</h3>")
        sources_output = gr.Textbox(value='', interactive=False, visible=False, show_label=False)
        #sources_output = []
        #for i in range(num_sources):
        #  source_elem = gr.HTML(visible=False)
        #  sources_output.append(source_elem)

  #define the behavior of prompt button later since it depends on user_chat_input
  prompt_button.click(prompt_select,
                      inputs=[question_type, number_of_questions, sa_desired_length],
                      outputs=[premade_prompt_output, user_chat_input])

  # Display input and output in three-ish parts
  # (using asynchronous functions):
  # First show user input, then show model output when complete
  # Then wait until the bot provides response and return the result
  # Finally, allow the user to ask a new question by reenabling input
  async_response = user_chat_submit.click(add_user_message,
            [user_chat_input, study_tutor],
            [user_chat_input, chatbot, study_tutor], queue=False) \
  .then(get_tutor_reply, [study_tutor], [user_chat_input, sources_output, chatbot, study_tutor], queue=True)

  async_response_b = user_chat_input.submit(add_user_message,
            [user_chat_input, study_tutor],
            [user_chat_input, chatbot, study_tutor], queue=False) \
  .then(get_tutor_reply, [study_tutor], [user_chat_input, sources_output, chatbot, study_tutor], queue=True)

  with gr.Box():
    gr.Markdown("""
    ## Export Your Chat History
    Export your chat history as a .json, PDF file, .txt, or .csv file
    """)
    with gr.Row():
      export_dialogue_button_json = gr.Button("JSON")
      export_dialogue_button_pdf = gr.Button("PDF")
      export_dialogue_button_txt = gr.Button("TXT")
      export_dialogue_button_csv = gr.Button("CSV")
    
    file_download = gr.Files(label="Download here",
                            file_types=['.pdf', '.txt', '.csv', '.json'], type="file", visible=False)
    
    export_dialogue_button_json.click(save_json, study_tutor, file_download, show_progress=True)
    export_dialogue_button_pdf.click(save_pdf, study_tutor, file_download, show_progress=True)
    export_dialogue_button_txt.click(save_txt, study_tutor, file_download, show_progress=True)
    export_dialogue_button_csv.click(save_csv, study_tutor, file_download, show_progress=True)


/tmp/ipykernel_62956/3581512034.py:98: GradioUnusedKwargWarning: You have unused kwarg parameters in Box, please remove them: {'scale': 1}
  with gr.Box(elem_id="sources-container", scale=1):


In [ ]:
FullStudyApp.queue()
FullStudyApp.launch(debug=True)

A little helper in case your ports are open and you just want to close them all. If this doesn't work, restart your IDE.

In [ ]:
gr.close_all()